In [1]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [2]:

# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%

background = ["Blue", "Orange", "Purple", "Red", "Yellow"] 
background_weights = [30, 40, 15, 5, 10]

bodies = ["Anasazi_Sweet", "Golden_Bantam", "Japanese_Black", "Teosinte", "Tesuque_Blue"] 
body_weights = [30, 40, 15, 5, 10]

heads = ["Cowboy", "Mario", "Naruto", "Pilgrim", "StPatrick"] 
head_weights = [30, 40, 15, 5, 10]

# Dictionary variable for each trait. 
# Eech trait corresponds to its file name

background_files = {
    "Blue": "blue",
    "Orange": "orange",
    "Purple": "purple",
    "Red": "red",
    "Yellow": "yellow",
}

body_files = {
    "Anasazi_Sweet": "anasazi_sweet",
    "Golden_Bantam": "golden_bantam",
    "Japanese_Black": "japanese_black_sticky",
    "Teosinte": "teosinte",
    "Tesuque_Blue": "tesuque_blue"   
}

head_files = {
    "Cowboy": "cowboy",
    "Mario": "mario",
    "Naruto": "naruto",
    "Pilgrim": "pilgrim",
    "StPatrick": "stpatrick"  
          
}

In [3]:
## Generate Traits

TOTAL_IMAGES = 30 # Number of random unique images we want to generate

all_images = [] 

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} #

    # For each trait category, select a random trait based on the weightings 
    new_image ["Background"] = random.choices(background, background_weights)[0]
    new_image ["Body"] = random.choices(bodies, body_weights)[0]
    new_image ["Head"] = random.choices(heads, head_weights)[0]
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)

In [4]:

# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [5]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [6]:
print(all_images)


[{'Background': 'Orange', 'Body': 'Japanese_Black', 'Head': 'StPatrick', 'tokenId': 0}, {'Background': 'Purple', 'Body': 'Japanese_Black', 'Head': 'Mario', 'tokenId': 1}, {'Background': 'Orange', 'Body': 'Golden_Bantam', 'Head': 'Mario', 'tokenId': 2}, {'Background': 'Blue', 'Body': 'Anasazi_Sweet', 'Head': 'Cowboy', 'tokenId': 3}, {'Background': 'Blue', 'Body': 'Golden_Bantam', 'Head': 'Cowboy', 'tokenId': 4}, {'Background': 'Orange', 'Body': 'Golden_Bantam', 'Head': 'Naruto', 'tokenId': 5}, {'Background': 'Purple', 'Body': 'Golden_Bantam', 'Head': 'Mario', 'tokenId': 6}, {'Background': 'Orange', 'Body': 'Golden_Bantam', 'Head': 'Pilgrim', 'tokenId': 7}, {'Background': 'Orange', 'Body': 'Japanese_Black', 'Head': 'Mario', 'tokenId': 8}, {'Background': 'Orange', 'Body': 'Japanese_Black', 'Head': 'Cowboy', 'tokenId': 9}, {'Background': 'Blue', 'Body': 'Tesuque_Blue', 'Head': 'Cowboy', 'tokenId': 10}, {'Background': 'Yellow', 'Body': 'Golden_Bantam', 'Head': 'Mario', 'tokenId': 11}, {'Bac

In [7]:

# Get Trait Counts

background_count = {}
for item in background:
    background_count[item] = 0
    
body_count = {}
for item in bodies:
    body_count[item] = 0

head_count = {}
for item in heads:
    head_count[item] = 0

for image in all_images:
    background_count[image["Background"]] += 1
    body_count[image["Body"]] += 1
    head_count[image["Head"]] += 1
    
print(background_count)
print(body_count)
print(head_count)

{'Blue': 9, 'Orange': 9, 'Purple': 5, 'Red': 2, 'Yellow': 5}
{'Anasazi_Sweet': 6, 'Golden_Bantam': 12, 'Japanese_Black': 8, 'Teosinte': 0, 'Tesuque_Blue': 4}
{'Cowboy': 10, 'Mario': 12, 'Naruto': 5, 'Pilgrim': 1, 'StPatrick': 2}


In [8]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [9]:
#### Generate Images    
for item in all_images:

    im1 = Image.open(f'./trait-layers/backgrounds/{background_files[item["Background"]]}.jpg').convert('RGBA')
    im2 = Image.open(f'./trait-layers/bodies/{body_files[item["Body"]]}.png').convert('RGBA')
    im3 = Image.open(f'./trait-layers/heads/{head_files[item["Head"]]}.png').convert('RGBA')

    #Create each composite
    com1 = Image.alpha_composite(im1, im2)
    com2 = Image.alpha_composite(com1, im3)

    #Convert to RGB
    rgb_im = com2.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)

In [10]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Body", i["Body"]))
    token["attributes"].append(getAttribute("Head", i["Head"]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()